# LangChain: Models, Prompts and Output Parsers

https://learn.deeplearning.ai/courses/langchain/lesson/2/models,-prompts-and-parsers

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [6]:
# !pip install python-dotenv
# !pip install openai
# !pip install --upgrade langchain

In [41]:
import os
import openai
from IPython.display import display, Markdown # to see better the output text

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [2]:
from openai import OpenAI

client = OpenAI()


def get_completion(prompt, model="gpt-3.5-turbo"):
    # completion = client.completions.create(model="davinci-002", prompt="Hello world")
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [4]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [3]:
customer_email = (
"Arrr, I be fuming that me blender lid "
"flew off and splattered me kitchen walls "
"with smoothie! And to make matters worse,"
"the warranty don't cover the cost of "
"cleaning up me kitchen. I need yer help right now, matey"
)

style = """American English in a calm and respectful tone"""

In [16]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey```



In [34]:
response = get_completion(prompt)
response

"Ah, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [14]:
import langchain
langchain.__version__

'0.2.10'

### Model

In [15]:
# ! pip install langchain_openai
from langchain_openai import ChatOpenAI

# To control the randomness and creativity of the generated

# text by an LLM, use temperature = 0.0

chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002436BC383D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002436BC39CF0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

### Prompt template

In [21]:
template_string = ("Translate the text that is delimited by triple backticks into"
                   " a style that is {style}. "
                   "text: ```{text}```")

In [22]:
from langchain_core.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate.from_template(template_string)

In [23]:
prompt_template.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```'))

In [24]:
prompt_template.messages[0].input_variables

['style', 'text']

In [26]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [27]:
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```")]

In [28]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [29]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)
customer_response

AIMessage(content="I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 88, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-35ccc03e-4099-4c98-84a7-0205b2b96b58-0', usage_metadata={'input_tokens': 88, 'output_tokens': 50, 'total_tokens': 138})

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [64]:
{"gift": False, "delivery_days": 5, "price_value": "pretty affordable!"}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [30]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [31]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [32]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [35]:
type(response.content)

str

You will get an error by running `response.content.get('gift')` because it's just a Python string

### Parse the LLM output string into a Python dictionary

In [37]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.",
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1.",
)



price_value_schema = ResponseSchema(
    name="price_value",

    description="Extract any sentences about the value or price, and output them as a comma separated Python list.",
)



response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [44]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
Markdown(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```

In [45]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)

In [48]:
Markdown(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [49]:
response = chat(messages)

In [50]:
print(response.content)

```json
{
	"gift": false,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [51]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False,
 'delivery_days': 2,
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [52]:
type(output_dict)

dict

## Concluding remarks

### We need to use Prompting to build advanced applications for many reasons:

* Prompts can be long and detailed 

* Prompt Templates are useful abstraction to reuse when you can 
* Prompting allow you to execute some common operations such as summarization, or question-answering, or connecting to SQL database or connectiong to API 

* Langchain Prompts supports output parsing to obtain the results in a specific format like json